# AdsorbML tutorial

The [AdsorbML](https://arxiv.org/abs/2211.16486) paper showed that pre-trained machine learning potentials were now viable to find and prioritize the best adsorption sites for a given surface. The results were quite impressive, especially if you were willing to do a DFT single-point calculation on the best calculations.

The latest UMA models are now total-energy models, and the results for the adsorption energy are even more impressive ([see the paper for details and benchmarks](https://ai.meta.com/research/publications/uma-a-family-of-universal-models-for-atoms/)). The AdsorbML package helps you with automated multi-adsorbate placement, and will automatically run calculations using the ML models to find the best sites to sample.

````{admonition} Need to install fairchem-core or get UMA access or getting permissions/401 errors?
:class: dropdown


1. Install the necessary packages using pip, uv etc
```{code-cell} ipython3
:tags: [skip-execution]

! pip install fairchem-core fairchem-data-oc fairchem-applications-cattsunami
```

2. Get access to any necessary huggingface gated models
    * Get and login to your Huggingface account
    * Request access to https://huggingface.co/facebook/UMA
    * Create a Huggingface token at https://huggingface.co/settings/tokens/ with the permission "Permissions: Read access to contents of all public gated repos you can access"
    * Add the token as an environment variable using `huggingface-cli login` or by setting the HF_TOKEN environment variable.

```{code-cell} ipython3
:tags: [skip-execution]

# Login using the huggingface-cli utility
! huggingface-cli login

# alternatively,
import os
os.environ['HF_TOKEN'] = 'MY_TOKEN'
```

````

## Define desired adsorbate+slab system

In [1]:
from __future__ import annotations

import pandas as pd
from fairchem.data.oc.core import Adsorbate, Bulk, Slab

bulk_src_id = "mp-30"
adsorbate_smiles = "*CO"

bulk = Bulk(bulk_src_id_from_db=bulk_src_id)
adsorbate = Adsorbate(adsorbate_smiles_from_db=adsorbate_smiles)
slabs = Slab.from_bulk_get_specific_millers(bulk=bulk, specific_millers=(1, 1, 1))

# There may be multiple slabs with this miller index.
# For demonstrative purposes we will take the first entry.
slab = slabs[0]

## Run heuristic/random adsorbate placement and ML relaxations

Now that we've defined the bulk, slab, and adsorbates of interest, we can quickly use the pre-trained UMA model as a calculator and the helper script `fairchem.core.components.calculate.recipes.adsorbml.run_adsorbml`. More details on the automated pipeline can be found at https://github.com/facebookresearch/fairchem/blob/main/src/fairchem/core/components/calculate/recipes/adsorbml.py#L316.

In [2]:
from ase.optimize import LBFGS
from fairchem.core import FAIRChemCalculator, pretrained_mlip
from fairchem.core.components.calculate.recipes.adsorbml import run_adsorbml

predictor = pretrained_mlip.get_predict_unit("uma-s-1p1")
calc = FAIRChemCalculator(predictor, task_name="oc20")

outputs = run_adsorbml(
    slab=slab,
    adsorbate=adsorbate,
    calculator=calc,
    optimizer_cls=LBFGS,
    fmax=0.02,
    steps=20,  # Increase to 200 for practical application, 20 is used for demonstrations
    num_placements=10,  # Increase to 100 for practical application, 10 is used for demonstrations
    reference_ml_energies=True,  # True if using a total energy model (i.e. UMA)
    relaxed_slab_atoms=None,
    place_on_relaxed_slab=False,
)

       Step     Time          Energy          fmax
LBFGS:    0 20:23:08     -300.219297        0.047121
LBFGS:    1 20:23:08     -300.219793        0.045099


LBFGS:    2 20:23:09     -300.225584        0.004437


/home/runner/work/_tool/Python/3.12.12/x64/lib/python3.12/site-packages/fairchem/data/oc/core/adsorbate.py:89: UserWarning: An adsorbate with that SMILES string was not found. Choosing one at random instead.
  warnings.warn(


       Step     Time          Energy          fmax
LBFGS:    0 20:23:09     -329.009720        2.969513
LBFGS:    1 20:23:09     -329.262831        1.461636


LBFGS:    2 20:23:10     -329.394579        0.970979
LBFGS:    3 20:23:10     -329.437779        0.865683


...

LBFGS:   11 20:23:49     -329.756327        0.529020
LBFGS:   12 20:23:49     -329.783692        0.643070


LBFGS:   13 20:23:49     -329.821875        0.804466
LBFGS:   14 20:23:50     -329.866311        0.804870


LBFGS:   15 20:23:50     -329.906365        0.506069
LBFGS:   16 20:23:50     -329.928643        0.354960


LBFGS:   17 20:23:50     -329.945221        0.405427
LBFGS:   18 20:23:50     -329.975022        0.566533


LBFGS:   19 20:23:50     -330.004452        0.510563
LBFGS:   20 20:23:51     -330.020985        0.304491


In [3]:
top_candidates = outputs["adslabs"]
global_min_candidate = top_candidates[0]

In [4]:
top_candidates = outputs["adslabs"]
pd.DataFrame(top_candidates)

,input_atoms,atoms,results
0,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.1717896940105, 'forces': [[0.0..."
1,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.08534674662286, 'forces': [[0...."
2,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.0244107725017, 'forces': [[0.0..."
3,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.0216680051677, 'forces': [[0.0..."
4,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.0209851743572, 'forces': [[0.0..."
5,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -330.0113912107342, 'forces': [[0.0..."
6,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -329.99206023234063, 'forces': [[0...."
7,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -329.9853482725017, 'forces': [[0.0..."
8,"{'atoms': (Atom('Cu', [np.float64(-1.300046521...","(Atom('Cu', [np.float64(-1.3000465215529715), ...","{'energy': -329.97815184611017, 'forces': [[0...."


## Write VASP input files

If you want to verify the results, you should run VASP. This assumes you have access to VASP pseudopotentials. The default VASP flags (which are equivalent to those used to make OC20) are located in `ocdata.utils.vasp`. Alternatively, you may pass your own vasp flags to the `write_vasp_input_files` function as `vasp_flags`. Note that to run this you need access to the VASP pseudopotentials and need to have those set up in ASE.

In [ ]:
import os

from fairchem.data.oc.utils.vasp import write_vasp_input_files

# Grab the 5 systems with the lowest energy
top_5_candidates = top_candidates[:5]

# Write the inputs
for idx, config in enumerate(top_5_candidates):
    os.makedirs(f"data/{idx}", exist_ok=True)
    write_vasp_input_files(config["atoms"], outdir=f"data/{idx}/")